In [60]:
import pandas as pd
import numpy as np

# Load datasets
df1 = pd.read_csv("Raw/dataset1.csv")
df2 = pd.read_csv("Raw/dataset2.csv")

# Preview
print("Dataset 1 head:")
display(df1.head())
print("Dataset 2 head:")
display(df2.head())


Dataset 1 head:


,start_time,bat_landing_to_food,habit,rat_period_start,rat_period_end,seconds_after_rat_arrival,risk,reward,month,sunset_time,hours_after_sunset,season
0,30/12/2017 18:37,16.000000,rat,30/12/2017 18:35,30/12/2017 18:38,108,1,0,0,30/12/2017 16:45,1.870833,0
1,30/12/2017 19:51,0.074016,fast,30/12/2017 19:50,30/12/2017 19:55,17,0,1,0,30/12/2017 16:45,3.100833,0
2,30/12/2017 19:51,4.000000,fast,30/12/2017 19:50,30/12/2017 19:55,41,0,1,0,30/12/2017 16:45,3.107500,0
3,30/12/2017 19:52,10.000000,rat,30/12/2017 19:50,30/12/2017 19:55,111,1,0,0,30/12/2017 16:45,3.126944,0
4,30/12/2017 19:54,15.000000,rat,30/12/2017 19:50,30/12/2017 19:55,194,1,0,0,30/12/2017 16:45,3.150000,0


Dataset 2 head:


,time,month,hours_after_sunset,bat_landing_number,food_availability,rat_minutes,rat_arrival_number
0,26/12/2017 16:13,0,-0.5,20,4.000000,0.0,0
1,26/12/2017 16:43,0,0.0,28,4.000000,0.0,0
2,26/12/2017 17:13,0,0.5,25,4.000000,0.0,0
3,26/12/2017 17:43,0,1.0,71,4.000000,0.0,0
4,26/12/2017 18:13,0,1.5,44,3.753857,0.0,0


In [61]:

# DATA QUALITY ASSESSMENT
print("=== INITIAL DATA ASSESSMENT ===")
print(f"Dataset 1: {df1.shape[0]} rows, {df1.shape[1]} columns")
print(f"Dataset 2: {df2.shape[0]} rows, {df2.shape[1]} columns")

# Store original metrics for comparison
original_df1_shape = df1.shape
original_df2_shape = df2.shape
original_df1_nulls = df1.isnull().sum().sum()
original_df2_nulls = df2.isnull().sum().sum()

print(f"\nMissing values - Dataset 1: {original_df1_nulls}")
print(f"Missing values - Dataset 2: {original_df2_nulls}")

print(f"\nDuplicates - Dataset 1: {df1.duplicated().sum()}")
print(f"Duplicates - Dataset 2: {df2.duplicated().sum()}")

=== INITIAL DATA ASSESSMENT ===
Dataset 1: 907 rows, 12 columns
Dataset 2: 2123 rows, 7 columns

Missing values - Dataset 1: 41
Missing values - Dataset 2: 0

Duplicates - Dataset 1: 1
Duplicates - Dataset 2: 0


In [62]:
def clean_columns(df):
    df.columns = (
        df.columns.astype(str)
                  .str.strip()
                  .str.lower()
                  .str.replace(r"[^\w\s]", "", regex=True)
                  .str.replace(r"\s+", "_", regex=True)
    )
    return df

df1 = clean_columns(df1)
df2 = clean_columns(df2)


In [63]:
for col in df1.select_dtypes(include="object").columns:
    df1[col] = df1[col].astype(str).str.strip()

for col in df2.select_dtypes(include="object").columns:
    df2[col] = df2[col].astype(str).str.strip()

    

In [64]:
# Date conversion
# Dataset1
for col in ["start_time","rat_period_start","rat_period_end","sunset_time"]:
    if col in df1.columns:
        df1[col] = pd.to_datetime(df1[col], errors="coerce", dayfirst=True)

# Dataset2
if "time" in df2.columns:
    df2["time"] = pd.to_datetime(df2["time"], errors="coerce", dayfirst=True)


In [65]:
# Numeric conversion
# Dataset1 numeric columns
for col in ["bat_landing_to_food","seconds_after_rat_arrival","risk","reward","hours_after_sunset"]:
    if col in df1.columns:
        df1[col] = pd.to_numeric(df1[col], errors="coerce")

# Dataset2 numeric columns
for col in ["hours_after_sunset","bat_landing_number","food_availability","rat_minutes","rat_arrival_number"]:
    if col in df2.columns:
        df2[col] = pd.to_numeric(df2[col], errors="coerce")

        

In [66]:
# Remove duplicates
df1 = df1.drop_duplicates().reset_index(drop=True)
df2 = df2.drop_duplicates().reset_index(drop=True)



In [67]:
# DATA CONSISTENCY VALIDATION
print("=== CONSISTENCY CHECKS ===")

# Check date logic in Dataset 1
if 'start_time' in df1.columns and 'rat_period_start' in df1.columns:
    date_issues = df1[df1['start_time'] < df1['rat_period_start']].shape[0]
    print(f"Bat arrivals before rat period starts: {date_issues}")

# Check risk/reward exclusivity
if 'risk' in df1.columns and 'reward' in df1.columns:
    both_risk_reward = df1[(df1['risk'] == 1) & (df1['reward'] == 1)].shape[0]
    print(f"Records with both risk=1 AND reward=1: {both_risk_reward}")

# Check hours_after_sunset consistency
negative_hours_df1 = df1[df1['hours_after_sunset'] < 0].shape[0] if 'hours_after_sunset' in df1.columns else 0
negative_hours_df2 = df2[df2['hours_after_sunset'] < 0].shape[0] if 'hours_after_sunset' in df2.columns else 0
print(f"Negative hours_after_sunset - Dataset 1: {negative_hours_df1}, Dataset 2: {negative_hours_df2}")

=== CONSISTENCY CHECKS ===
Bat arrivals before rat period starts: 0
Records with both risk=1 AND reward=1: 98
Negative hours_after_sunset - Dataset 1: 12, Dataset 2: 212


In [68]:
# Data validation rules
# Risk / Reward must be 0 or 1
for col in ["risk","reward"]:
    if col in df1.columns:
        df1.loc[~df1[col].isin([0,1]), col] = np.nan

# Remove negative values
for col in ["bat_landing_to_food","seconds_after_rat_arrival","hours_after_sunset"]:
    if col in df1.columns:
        df1.loc[df1[col] < 0, col] = np.nan

for col in ["hours_after_sunset","rat_minutes"]:
    if col in df2.columns:
        df2.loc[df2[col] < 0, col] = np.nan

# Cap hours_after_sunset to 24h
for df in [df1, df2]:
    if "hours_after_sunset" in df.columns:
        df.loc[df["hours_after_sunset"] > 24, "hours_after_sunset"] = np.nan

        
        

In [69]:
# OUTLIER DETECTION
def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("=== OUTLIER ANALYSIS ===")
outlier_columns = ['bat_landing_to_food', 'seconds_after_rat_arrival', 'hours_after_sunset']

for col in outlier_columns:
    if col in df1.columns:
        outliers, lower, upper = detect_outliers_iqr(df1, col)
        print(f"{col}: {len(outliers)} outliers detected (bounds: {lower:.2f} - {upper:.2f})")
        # Optionally cap extreme outliers
        extreme_outliers = len(df1[df1[col] > upper + 2*(upper-lower)])
        if extreme_outliers > 0:
            print(f"  - {extreme_outliers} extreme outliers beyond 2*IQR")

=== OUTLIER ANALYSIS ===
bat_landing_to_food: 88 outliers detected (bounds: -15.12 - 27.88)
  - 12 extreme outliers beyond 2*IQR
seconds_after_rat_arrival: 0 outliers detected (bounds: -447.75 - 984.25)
hours_after_sunset: 0 outliers detected (bounds: -1.59 - 12.83)


In [70]:
# CLEANING IMPACT SUMMARY
final_df1_shape = df1.shape
final_df2_shape = df2.shape
final_df1_nulls = df1.isnull().sum().sum()
final_df2_nulls = df2.isnull().sum().sum()

print("=== CLEANING SUMMARY REPORT ===")
print(f"Dataset 1:")
print(f"  Rows: {original_df1_shape[0]} → {final_df1_shape[0]} (removed: {original_df1_shape[0] - final_df1_shape[0]})")
print(f"  Missing values: {original_df1_nulls} → {final_df1_nulls} (change: {final_df1_nulls - original_df1_nulls})")

print(f"\nDataset 2:")
print(f"  Rows: {original_df2_shape[0]} → {final_df2_shape[0]} (removed: {original_df2_shape[0] - final_df2_shape[0]})")
print(f"  Missing values: {original_df2_nulls} → {final_df2_nulls} (change: {final_df2_nulls - original_df2_nulls})")

=== CLEANING SUMMARY REPORT ===
Dataset 1:
  Rows: 907 → 906 (removed: 1)
  Missing values: 41 → 12 (change: -29)

Dataset 2:
  Rows: 2123 → 2123 (removed: 0)
  Missing values: 0 → 212 (change: 212)


In [71]:
# Data quality score
completeness_df1 = 1 - (final_df1_nulls / (final_df1_shape[0] * final_df1_shape[1]))
completeness_df2 = 1 - (final_df2_nulls / (final_df2_shape[0] * final_df2_shape[1]))

print(f"\nData Completeness Scores:")
print(f"  Dataset 1: {completeness_df1:.1%}")
print(f"  Dataset 2: {completeness_df2:.1%}")


Data Completeness Scores:
  Dataset 1: 99.9%
  Dataset 2: 98.6%


In [72]:
# Save cleaned datasets
df1.to_csv("dataset1_clean.csv", index=False)
df2.to_csv("dataset2_clean.csv", index=False)

print("Cleaning done. Files saved as dataset1_clean.csv and dataset2_clean.csv")



Cleaning done. Files saved as dataset1_clean.csv and dataset2_clean.csv


In [73]:
# FINAL DATA VALIDATION

print("=== FINAL VALIDATION CHECKS ===")

# Load and preview cleaned data
cleaned_df1 = pd.read_csv("dataset1_clean.csv")
cleaned_df2 = pd.read_csv("dataset2_clean.csv")
print(df1.head())   # first 5 rows of dataset1_clean
print(df2.head())   # first 5 rows of dataset2_clean

=== FINAL VALIDATION CHECKS ===
           start_time  bat_landing_to_food habit    rat_period_start  \
0 2017-12-30 18:37:00            16.000000   rat 2017-12-30 18:35:00   
1 2017-12-30 19:51:00             0.074016  fast 2017-12-30 19:50:00   
2 2017-12-30 19:51:00             4.000000  fast 2017-12-30 19:50:00   
3 2017-12-30 19:52:00            10.000000   rat 2017-12-30 19:50:00   
4 2017-12-30 19:54:00            15.000000   rat 2017-12-30 19:50:00   

       rat_period_end  seconds_after_rat_arrival  risk  reward  month  \
0 2017-12-30 18:38:00                      108.0   1.0     0.0      0   
1 2017-12-30 19:55:00                       17.0   0.0     1.0      0   
2 2017-12-30 19:55:00                       41.0   0.0     1.0      0   
3 2017-12-30 19:55:00                      111.0   1.0     0.0      0   
4 2017-12-30 19:55:00                      194.0   1.0     0.0      0   

          sunset_time  hours_after_sunset  season  
0 2017-12-30 16:45:00            1.870833   

In [74]:
# Basic integrity checks
assert cleaned_df1.shape[0] > 0, "Dataset 1 cannot be empty"
assert cleaned_df2.shape[0] > 0, "Dataset 2 cannot be empty"

In [75]:
# Check data types
object_cols_df1 = cleaned_df1.select_dtypes(include=['object']).columns.tolist()
print(f"Object columns in Dataset 1: {object_cols_df1}")

Object columns in Dataset 1: ['start_time', 'habit', 'rat_period_start', 'rat_period_end', 'sunset_time']


In [76]:
# Flexible validation - check that expected columns exist
expected_categorical = ['habit']
for col in expected_categorical:
    if col in cleaned_df1.columns:
        print(f"✓ {col} column present in dataset")
    else:
        print(f"⚠ {col} column missing from dataset")

# Value range checks
if 'hours_after_sunset' in test_df1.columns:
    max_hours = cleaned_df1['hours_after_sunset'].max()
    if pd.notna(max_hours) and max_hours <= 24:
        print("✓ Hours after sunset within valid range")
    else:
        print(f"⚠ Max hours after sunset: {max_hours}")

✓ habit column present in dataset
✓ Hours after sunset within valid range


In [77]:
# Risk/reward validation
if 'risk' in cleaned_df1.columns:
    valid_risk = cleaned_df1['risk'].dropna().isin([0, 1]).all()
    print(f"✓ Risk values valid: {valid_risk}")

if 'reward' in cleaned_df1.columns:
    valid_reward = cleaned_df1['reward'].dropna().isin([0, 1]).all()
    print(f"✓ Reward values valid: {valid_reward}")

print("All validation checks completed!")
print("Data is ready for Visualization and Analysis")

✓ Risk values valid: True
✓ Reward values valid: True
All validation checks completed!
Data is ready for Visualization and Analysis


In [78]:
df1.head()
df2.head()

,time,month,hours_after_sunset,bat_landing_number,food_availability,rat_minutes,rat_arrival_number
0,2017-12-26 16:13:00,0,NaN,20,4.000000,0.0,0
1,2017-12-26 16:43:00,0,0.0,28,4.000000,0.0,0
2,2017-12-26 17:13:00,0,0.5,25,4.000000,0.0,0
3,2017-12-26 17:43:00,0,1.0,71,4.000000,0.0,0
4,2017-12-26 18:13:00,0,1.5,44,3.753857,0.0,0
